In [1]:
import pandas as pd

In [2]:
reach_df = pd.read_csv('../../learn/summary/reach.csv')
reach_df.head(2)

,compound_id,disease_id
0,Q407431,Q815819
1,Q407431,Q377978


In [3]:
diseases = reach_df['disease_id'].unique()

In [4]:
node_neo = pd.read_csv('data/hetnet_nodes.csv')
node_neo.head(2)

,:ID,identifier:String,name:String,:LABEL
0,Q24778004,Q24778004,Spectrin repeat,Structural Motif
1,Q24774294,Q24774294,Furin-like repeat,Structural Motif


In [5]:
len(node_neo)

69077

In [6]:
# Can't query for columns with a : in the label name
node_neo["ID"] = node_neo[":ID"]
node_neo["LABEL"] = node_neo[':LABEL']

In [7]:
to_remove = node_neo.query('LABEL == "Disease" and ID not in @diseases')['ID'].values

In [8]:
node_neo = (node_neo.query('ID not in @to_remove')
                .reset_index(drop=True)
                .drop(['ID', 'LABEL'], axis=1))
node_neo.head(2)

,:ID,identifier:String,name:String,:LABEL
0,Q24778004,Q24778004,Spectrin repeat,Structural Motif
1,Q24774294,Q24774294,Furin-like repeat,Structural Motif


In [9]:
len(node_neo)

67602

In [10]:
edge_neo = pd.read_csv('data/hetnet_edges.csv')
edge_neo.head(2)

,:START_ID,:END_ID,:TYPE
0,Q24778004,Q24768156,part-of_SMpoPF
1,Q24778004,Q24768109,part-of_SMpoPF


In [11]:
len(edge_neo)

693773

In [12]:
edge_neo['sid'] = edge_neo[':START_ID']
edge_neo['eid'] = edge_neo[':END_ID']

In [13]:
edge_neo = (edge_neo.query('sid not in @to_remove and eid not in @to_remove')
                .reset_index(drop=True)
                .drop(['sid', 'eid'], axis=1))
len(edge_neo)

692394

In [14]:
node_neo.to_csv('../neo/import/hetnet_nodes.csv', index=False)
edge_neo.to_csv('../neo/import/hetnet_edges.csv', index=False)

In [15]:
import sys
sys.path.append('../py')
from permute import run_permute_pipeline

In [16]:
run_permute_pipeline('../neo/import/hetnet_edges.csv', save_dir='../neo/import')

Reading Edges...
Starting permutation 1
Saved file:
/home/mmayers/projects/wikidata/neo/import/hetnet_edges_perm-1.csv

Starting permutation 2
Saved file:
/home/mmayers/projects/wikidata/neo/import/hetnet_edges_perm-2.csv

Starting permutation 3
Saved file:
/home/mmayers/projects/wikidata/neo/import/hetnet_edges_perm-3.csv

Starting permutation 4
Saved file:
/home/mmayers/projects/wikidata/neo/import/hetnet_edges_perm-4.csv

Starting permutation 5
Saved file:
/home/mmayers/projects/wikidata/neo/import/hetnet_edges_perm-5.csv

Saved statistics on permutations to:
/home/mmayers/projects/wikidata/neo/import/stats.tsv
Took 0 hours, 16 minutes, 8 seconds


In [16]:
import json
from processing import prep_hetio

In [17]:
edges = pd.read_hdf('data/prepped_edges.h5')
with open('data/node_types.json') as fin:
    node_types = json.load(fin)

reciprocals = []
with open('data/reciprocal_relations.txt', 'r') as fin2:
    for line in fin2:
        reciprocals.append(line.rstrip(',\n').split(','))

In [18]:
to_remove_full = 'http://www.wikidata.org/entity/' + to_remove

In [19]:
edges = edges.query('s not in @to_remove_full and o not in @to_remove_full').reset_index(drop=True)

In [32]:
prep_hetio(edges, node_types, reciprocals, neo_nodes=node_neo, neo_edges=edge_neo)

Added 67602 nodes to graph
Added 692394 edges to graph
